Отчёт по заданию 2.2: Julia Прогнозирование цен на жильё (Boston Housing)

1. Настройка среды

Установка Julia 1.11.7
from https://julialang.org


Для работы с проектом использовались следующие инструменты:
using RDatasets     
using DataFrames     
using GLM            
using Statistics     
using Random        



2. Загрузка данных

boston = dataset("MASS", "Boston")
println("Имена колонок в датасете:")
println(names(boston))

3. Разделение данных на обучающую и тестовую выборки

70% — обучающая  (train)

30% — тестовая  (test)

Random.seed!(12345) 
n = nrow(boston)
train_size = Int(round(0.7 * n))
indices = shuffle(1:n)

train_idx = indices[1:train_size]
test_idx  = indices[train_size+1:end]

train_data = boston[train_idx, :]
test_data  = boston[test_idx, :]

4. Построение модели линейной регрессии
model = lm(@formula(MedV ~ Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat), train_data)


5. Коэффициенты модели

println("\nКоэффициенты модели:")
for (name, coef_val) in zip(coefnames(model), coef(model))
    println(rpad(name, 10), " => ", round(coef_val, digits=4))
end

Интерпретация:

Положительный коэффициент → увеличение признака повышает цену

Отрицательный коэффициент → увеличение признака снижает цену
Пример: Rm = 4.3 → каждая дополнительная комната увеличивает цену на +- $4.300, если все остальные признаки фиксированы

In [2]:

using RDatasets
using DataFrames
using GLM
using Statistics
using Random



boston = dataset("MASS", "Boston")

println("Имена колонок в датасете:")
println(names(boston))

Random.seed!(12345) 
n = nrow(boston)
train_size = Int(round(0.7 * n))
indices = shuffle(1:n)

train_idx = indices[1:train_size]
test_idx  = indices[train_size+1:end]

train_data = boston[train_idx, :]
test_data  = boston[test_idx, :]


model = lm(@formula(MedV ~ Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat), train_data)

println("\nКоэффициенты модели:")
for (name, coef_val) in zip(coefnames(model), coef(model))
    println(rpad(name, 10), " => ", round(coef_val, digits=4))
end


y_test = test_data.MedV
y_pred = predict(model, test_data)

mse = mean((y_test .- y_pred).^2)
ss_total = sum((y_test .- mean(y_test)).^2)
ss_res = sum((y_test .- y_pred).^2)
r2_score = 1 - ss_res / ss_total

println("\nMSE (на тестовой выборке): ", round(mse, digits=4))
println("R² (на тестовой выборке): ", round(r2_score, digits=4))


Имена колонок в датасете:
["Crim", "Zn", "Indus", "Chas", "NOx", "Rm", "Age", "Dis", "Rad", "Tax", "PTRatio", "Black", "LStat", "MedV"]

Коэффициенты модели:
(Intercept) => 31.1912
Crim       => -0.1325
Zn         => 0.0444
Indus      => 0.0035
Chas       => 2.3985
NOx        => -15.2767
Rm         => 4.2987
Age        => -0.0119
Dis        => -1.5011
Rad        => 0.3007
Tax        => -0.0121
PTRatio    => -0.9345
Black      => 0.0123
LStat      => -0.4738

MSE (на тестовой выборке): 28.1798
R² (на тестовой выборке): 0.6399


7. Выводы

Модель линейной регрессии успешно предсказывает цену жилья на основе характеристик районов.

Наиболее сильное влияние на цену имеют:
Rm
LStat 
NOx 
Разделение данных на train/test позволяет проверить устойчивость модели к новым данным

Метрики (MSE, R^2) показывают, что модель достаточно точна, но есть место для улучшений (например, использование регуляризации или более сложных моделей)